In [2]:
import os
import numpy as np
import pandas as pd
import json
import cirq
import matplotlib.pyplot as plt
from scipy.optimize import minimize

def load_historical_stock_data(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    stocks = list(data.keys())[15:25]
    prices = []
    for stock in stocks:
        prices.append([entry['close'] for entry in data[stock]])
    prices = np.array(prices).T
    return pd.DataFrame(prices, columns=stocks)

stock_prices = load_historical_stock_data('historical.json')
num_stocks = stock_prices.shape[1]
num_days = stock_prices.shape[0]

normalized_stock_prices = stock_prices / stock_prices.iloc[0]

def cost(weights, returns, risk_factors):
    portfolio_return = np.dot(weights, returns.mean(axis=0))
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(np.cov(returns, rowvar=False), weights)))
    weighted_risk_factor = np.dot(weights, risk_factors)
    return -portfolio_return + portfolio_risk + 0.1 * weighted_risk_factor

def construct_adjacency_matrix(returns):
    covariance_matrix = np.cov(returns, rowvar=False)
    max_cov = np.max(covariance_matrix)
    if max_cov != 0:
        adjacency_matrix = covariance_matrix / max_cov
    else:
        adjacency_matrix = covariance_matrix
    return adjacency_matrix

def quantum_walk_on_graph(qubits, adjacency_matrix, steps):
    circuit = cirq.Circuit()
    num_qubits = len(qubits)

    for qubit in qubits:
        circuit.append(cirq.H(qubit))
    
    for _ in range(steps):
        for i in range(num_qubits):
            for j in range(num_qubits):
                if i != j and adjacency_matrix[i, j] > 0:
                    circuit.append(cirq.CZ(qubits[i], qubits[j]) ** adjacency_matrix[i, j])
        for i in range(num_qubits):
            circuit.append(cirq.H(qubits[i]))
    circuit.append(cirq.measure(*qubits, key='result'))
    return circuit

def qaoa_circuit(qubits, beta, gamma):
    circuit = cirq.Circuit()
    for j in range(len(qubits)):
        circuit.append(cirq.rx(2 * beta[j])(qubits[j]))
    for i in range(len(qubits) - 1):
        circuit.append(cirq.CNOT(qubits[i], qubits[i + 1]))
    
    for j in range(len(qubits)):
        circuit.append(cirq.rx(2 * gamma[j])(qubits[j]))
    
    circuit.append(cirq.measure(*qubits, key='result'))
    
    return circuit

def optimize_portfolio(stock_prices, total_investment, steps=3):
    returns = stock_prices.pct_change().dropna().values
    num_stocks = returns.shape[1]

    fluctuation = stock_prices.pct_change().std().values
    risk_factors = fluctuation / np.max(fluctuation)

    print("Risk Factors for each stock:")
    for stock, risk in zip(stock_prices.columns, risk_factors):
        print(f"{stock}: {risk:.2f}")

    adjacency_matrix = construct_adjacency_matrix(returns)

    qubits = [cirq.NamedQubit(f'Stock_{i+1}') for i in range(num_stocks)]
    
    def qaoa_objective(params):
        beta = params[:num_stocks]
        gamma = params[num_stocks:]
        qaoa_circuit_instance = qaoa_circuit(qubits, beta, gamma)
        sim = cirq.Simulator()
        qaoa_result = sim.run(qaoa_circuit_instance, repetitions=100)
        qaoa_counts = qaoa_result.measurements['result']
        qaoa_sampled_weights = np.zeros(num_stocks)
        for sample in qaoa_counts:
            index = int("".join(str(x) for x in sample), 2) % num_stocks
            qaoa_sampled_weights[index] += 1
        qaoa_sampled_weights = qaoa_sampled_weights / np.sum(qaoa_sampled_weights)
        return cost(qaoa_sampled_weights, returns, risk_factors)

    initial_params = np.random.rand(2 * num_stocks)
    result = minimize(qaoa_objective, initial_params, method='COBYLA')
    optimized_params = result.x
    beta = optimized_params[:num_stocks]
    gamma = optimized_params[num_stocks:]

    walk_circuit = quantum_walk_on_graph(qubits, adjacency_matrix, steps)
    sim = cirq.Simulator()
    walk_result = sim.run(walk_circuit, repetitions=100)
    
    qaoa_circuit_instance = qaoa_circuit(qubits, beta, gamma)
    qaoa_result = sim.run(qaoa_circuit_instance, repetitions=100)
    
    walk_counts = walk_result.measurements['result']
    walk_sampled_weights = np.zeros(num_stocks)
    for sample in walk_counts:
        index = int("".join(str(x) for x in sample), 2) % num_stocks
        walk_sampled_weights[index] += 1

    qaoa_counts = qaoa_result.measurements['result']
    qaoa_sampled_weights = np.zeros(num_stocks)
    for sample in qaoa_counts:
        index = int("".join(str(x) for x in sample), 2) % num_stocks
        qaoa_sampled_weights[index] += 1

    combined_weights = (walk_sampled_weights + qaoa_sampled_weights) / 2
    final_weights = combined_weights / np.sum(combined_weights)

    investment_amounts = final_weights * total_investment
    return final_weights, investment_amounts, walk_sampled_weights, qaoa_sampled_weights

total_investment = 10000
optimized_weights, investment_amounts, walk_sampled_weights, qaoa_sampled_weights = optimize_portfolio(stock_prices, total_investment, steps=3)

if not os.path.exists('graphs_new_test1'):
    os.makedirs('graphs_new_test1')

plt.figure(figsize=(12, 6))
plt.plot(normalized_stock_prices)
plt.title('Normalized Historical Stock Prices Over Time')
plt.xlabel('Days')
plt.ylabel('Normalized Price')
plt.legend(stock_prices.columns, bbox_to_anchor=(1.05, 1), loc='upper left')
plt.grid()
plt.tight_layout()
plt.savefig('graphs_new_test1/normalized_historical_stock_prices.png')
plt.close()

plt.figure(figsize=(12, 6))
plt.bar(stock_prices.columns, walk_sampled_weights / np.sum(walk_sampled_weights))
plt.title('Sampled Weights from Quantum Walk')
plt.xlabel('Stocks')
plt.ylabel('Normalized Weight')
plt.xticks(rotation=45, ha='right')
plt.grid()
plt.tight_layout()
plt.savefig('graphs_new_test1/quantum_walk_weights.png')
plt.close()

plt.figure(figsize=(12, 6))
plt.bar(stock_prices.columns, qaoa_sampled_weights / np.sum(qaoa_sampled_weights))
plt.title('Sampled Weights from QAOA')
plt.xlabel('Stocks')
plt.ylabel('Normalized Weight')
plt.xticks(rotation=45, ha='right')
plt.grid()
plt.tight_layout()
plt.savefig('graphs_new_test1/qaoa_weights.png')
plt.close()

plt.figure(figsize=(8, 8))
plt.pie(investment_amounts, labels=stock_prices.columns, autopct='%1.1f%%', startangle=140)
plt.title('Final Investment Distribution')
plt.axis('equal')
plt.tight_layout()
plt.savefig('graphs_new_test1/investment_distribution.png')
plt.close()

plt.figure(figsize=(12, 6))
plt.bar(stock_prices.columns, optimized_weights)
plt.title('Optimized Portfolio Weights')
plt.xlabel('Stocks')
plt.ylabel('Weights')
plt.xticks(rotation=45, ha='right')
plt.grid()
plt.tight_layout()
plt.savefig('graphs_new_test1/optimized_portfolio_weights.png')
plt.close()

Risk Factors for each stock:
SBI LIFE INSURANCE CO LTD: 0.56
BAJAJ FINANCE LIMITED: 0.59
LARSEN & TOUBRO LTD.: 0.65
BRITANNIA INDUSTRIES LTD: 0.44
BAJAJ AUTO LIMITED: 0.57
NTPC LTD: 0.77
TITAN COMPANY LIMITED: 0.53
ADANI ENTERPRISES LIMITED: 0.98
ADANI PORT & SEZ LTD: 1.00
SUN PHARMACEUTICAL IND L: 0.43
